# Search Engine

CREDS : I hereby thanks pyimagesearch for this awesome tutorial which helped me build this from his blog https://www.pyimagesearch.com/2014/01/27/hobbits-and-histograms-a-how-to-guide-to-building-your-first-image-search-engine-in-python/


### The 4 Steps to Building an Image Search Engine
On the most basic level, there are four steps to building an image search engine:

1. Define your descriptor: What type of descriptor are you going to use? Are you describing color? Texture? Shape?
2. Index your dataset: Apply your descriptor to each image in your dataset, extracting a set of features.
3. Define your similarity metric: How are you going to define how “similar” two images are? You’ll likely be using some sort of distance metric. Common choices include Euclidean, Cityblock (Manhattan), Cosine, and chi-squared to name a few.
4. Searching: To perform a search, apply your descriptor to your query image, and then ask your distance metric to rank how similar your images are in your index to your query images. Sort your results via similarity and then examine them.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import imutils
import cv2
import os
from imutils.paths import list_images
import pickle

In [12]:
args = {
    "dataset" : "dataset/images/",
    "index" : "index.cpickle",
    "query" : "Mordor-001.png"
}

In [3]:
class RGBHistogram : 
    
    def __init__ (self,bins) :
        self.bins = bins
        
    def describe(self, image):
        # 3d histogram and processing it
        hist = cv2.calcHist([image], [0,1,2], None, self.bins, [0,256,0,256,0,256])
        # Normalizing it
        hist = cv2.normalize(hist, hist)
        return hist.flatten()
        

In [4]:
# Step 2 : Indexing our Dataset
# initialize the index dictionary to store our our quantifed
# images, with the 'key' of the dictionary being the image
# filename and the 'value' our computed features
index = {}
desc = RGBHistogram([8, 8, 8])

In [5]:
for imagepath in list_images(args["dataset"]):
#     Extracting image id
    k = imagepath[imagepath.rfind("/") + 1 :]
    image = cv2.imread(imagepath)
    features = desc.describe(image)
    index[k] = features
    

In [6]:
f = open(args["index"], "wb")
f.write(pickle.dumps(index))
f.close()
print("Done Indexing {} Images ".format(len(index)))

Done Indexing 25 Images 


In [7]:
# Step 3 : Searching
# Search on basis of similarities
class Searcher:
    
    def __init__(self, index):
        self.index = index
    
    def search(self, queryFeatures):
        results = {}
        for (k,features) in self.index.items():
            #We will use chi2Square Distance to compare similarity
            d = self.chi2distance(features, queryFeatures)
            results[k] = d
        # the smaller the chi-squared distance, the relevant/similar
        results = sorted([(v,k) for (k,v) in results.items()])
        return results
    
    def chi2distance(self, histA, histB, eps = 1e-10):
        d = 0.5 * np.sum([((a-b) ** 2) / (a+b+eps) for (a,b) in zip(histA,histB)])
        return d        

In [8]:
# Step 4 : Performing Search
index = pickle.loads(open(args["index"], "rb").read())
searcher = Searcher(index)

In [ ]:
for (query, queryFeatures) in index.items():
    results = searcher.search(queryFeatures)
    
    path = os.path.join(args["dataset"], query)
    queryImage = cv2.imread(path)
    cv2.imshow("Query", queryImage)
    print("query {} ".format(query))
    
    # Display Results in montages
    montageA = np.zeros((166*5, 400, 3), dtype = "uint8")
    montageB = np.zeros((166*5, 400, 3), dtype = "uint8")

    # Looping over top 10 results
    for j in range(0,10):
        (score, imageName) = results[j]
        path = os.path.join(args["dataset"], imageName)
        result = cv2.imread(path)
        print("\t{}. {} : {:.3f}".format(j + 1, imageName, score))
        # check to see if the first montage should be used
        if j < 5:
            montageA[j * 166:(j + 1) * 166, :] = result

        # otherwise, the second montage should be used
        else:
            montageB[(j - 5) * 166:((j - 5) + 1) * 166, :] = result

    # show the results
    cv2.imshow("Results 1-5", montageA)
    cv2.imshow("Results 6-10", montageB)
    cv2.waitKey(0)



query Dol-Guldur-005.png 
	1. Dol-Guldur-005.png : 0.000
	2. Dol-Guldur-001.png : 0.213
	3. Dol-Guldur-003.png : 0.325
	4. Dol-Guldur-004.png : 0.327
	5. Dol-Guldur-002.png : 0.477
	6. Goblin-001.png : 1.001
	7. Golbin-005.png : 1.134
	8. Mordor-005.png : 1.150
	9. Mordor-003.png : 1.194
	10. Goblin-002.png : 1.203
query Dol-Guldur-004.png 
	1. Dol-Guldur-004.png : 0.000
	2. Dol-Guldur-003.png : 0.042
	3. Dol-Guldur-001.png : 0.285
	4. Dol-Guldur-005.png : 0.327
	5. Dol-Guldur-002.png : 0.534
	6. Goblin-001.png : 0.548
	7. Golbin-005.png : 0.679
	8. Mordor-005.png : 0.732
	9. Goblin-002.png : 0.782
	10. Mordor-003.png : 0.802
